# Evaluation With PyTerrier

In [2]:
!pip install python-terrier

Defaulting to user installation because normal site-packages is not writeable


In [13]:
import pandas as pd
import pyterrier as pt
if not pt.started():
  pt.init()

def load_data(data_type, topics_to_exclude):
    import json
    from tqdm import tqdm
    if data_type not in ['PRO', 'CON', 'ONTOPIC']:
        raise ValueError('X')


    qrels = pd.read_csv('task3-qrels.txt', sep='\s', names=['qid', 'stance', 'docno', 'label'])
    qrels = qrels[qrels['stance'] == data_type]
    qrels = qrels[~qrels['qid'].astype(int).isin(topics_to_exclude)]
    qrels['qid'] = qrels['qid'].astype(str)

    topics = pd.DataFrame([{'qid': str(i), 'query': str(i)} for i in qrels.qid.unique()])

    return topics, qrels

def load_run(run_name, data_type, topics_to_exclude):
    if data_type not in ['PRO', 'CON', 'ONTOPIC']:
        raise ValueError('X')
    ret = pt.io.read_results(run_name)
    ret = pd.read_csv(run_name, names=['qid', 'q0', 'docno', 'rank', 'score', 'system'], sep=' ')
    ret = ret[~ret['qid'].astype(int).isin(topics_to_exclude)]
    
    if data_type != 'ONTOPIC':
        ret = ret[ret['q0'] == data_type]
    ret['qid'] = ret['qid'].astype(str)
  
    return pt.transformer.SourceTransformer(ret)

def load_report(stance, measure, models=['google', 'boromir-touche23', 'dummy', 'random', 'afinn', 'bert', 'formula',
          'oracle', 'V3-model_1', 'V6-model_all_topic_w0', 'V6-model_all_topic_w5',
          'V8-model_all_topic', 'V9-model_all_topic'], topics_to_exclude = [1, 2, 4, 8, 9, 10, 15, 20, 21, 22, 27, 31, 33, 36, 37, 40, 43, 45, 47, 48]):
    topics, qrels = load_data(stance,topics_to_exclude)
    runs = [load_run(f'../out/elastic/V3-model_1/{i}/run.txt', stance, topics_to_exclude) for i in models]

    return pt.Experiment(
        runs,
        topics,
        qrels,
        eval_metrics=[measure],
        names=models,
        baseline=0,
        correction='bonferroni'
    ).sort_values(measure, ascending=False)

### Evaluation with Precision@10

In [14]:
load_report('PRO', 'P_10')

/home/maik/workspace/ecir23-unjudged-documents/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


,name,P_10,P_10 +,P_10 -,P_10 p-value,P_10 reject,P_10 p-value corrected
7,oracle,1.000000,28.0,0.0,8.389231e-10,True,1.090600e-08
5,bert,0.713333,14.0,7.0,1.924885e-02,False,2.502351e-01
6,formula,0.706667,15.0,6.0,1.697733e-02,False,2.207053e-01
2,dummy,0.686667,13.0,7.0,7.049986e-02,False,9.164982e-01
9,V6-model_all_topic_w0,0.676667,11.0,8.0,1.675025e-01,False,1.000000e+00
3,random,0.666667,14.0,9.0,1.868092e-01,False,1.000000e+00
1,boromir-touche23,0.636667,15.0,10.0,6.772122e-01,False,1.000000e+00
0,google,0.623333,NaN,NaN,NaN,False,NaN
10,V6-model_all_topic_w5,0.620000,14.0,13.0,9.287943e-01,False,1.000000e+00
4,afinn,0.603333,9.0,13.0,5.651799e-01,False,1.000000e+00


In [15]:
load_report('CON', 'P_10')

/home/maik/workspace/ecir23-unjudged-documents/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


,name,P_10,P_10 +,P_10 -,P_10 p-value,P_10 reject,P_10 p-value corrected
7,oracle,0.800000,30.0,0.0,3.970421e-15,True,5.161547e-14
1,boromir-touche23,0.243333,19.0,6.0,7.544304e-03,False,9.807595e-02
5,bert,0.233333,19.0,6.0,5.822195e-03,False,7.568853e-02
10,V6-model_all_topic_w5,0.220000,19.0,7.0,1.262878e-02,False,1.641742e-01
9,V6-model_all_topic_w0,0.213333,15.0,8.0,3.458249e-02,False,4.495723e-01
4,afinn,0.193333,13.0,9.0,2.008691e-01,False,1.000000e+00
2,dummy,0.190000,15.0,10.0,2.363492e-01,False,1.000000e+00
8,V3-model_1,0.180000,10.0,9.0,3.431365e-01,False,1.000000e+00
6,formula,0.180000,13.0,10.0,2.861040e-01,False,1.000000e+00
3,random,0.160000,10.0,10.0,7.245199e-01,False,1.000000e+00


In [16]:
load_report('ONTOPIC', 'P_10')

/home/maik/workspace/ecir23-unjudged-documents/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


,name,P_10,P_10 +,P_10 -,P_10 p-value,P_10 reject,P_10 p-value corrected
7,oracle,1.000000,24.0,0.0,2.732014e-07,True,0.000004
3,random,0.916667,15.0,6.0,1.203230e-02,False,0.156420
2,dummy,0.913333,16.0,6.0,1.368112e-02,False,0.177855
5,bert,0.910000,14.0,5.0,1.003009e-02,False,0.130391
9,V6-model_all_topic_w0,0.906667,15.0,6.0,2.199830e-02,False,0.285978
6,formula,0.900000,13.0,5.0,2.642456e-02,False,0.343519
10,V6-model_all_topic_w5,0.893333,17.0,8.0,4.201138e-02,False,0.546148
1,boromir-touche23,0.890000,13.0,5.0,2.684599e-02,False,0.348998
4,afinn,0.886667,12.0,8.0,7.293518e-02,False,0.948157
8,V3-model_1,0.883333,14.0,5.0,8.307467e-02,False,1.000000


### Evaluation nDCG@10

In [17]:
load_report('PRO', 'ndcg_cut_10')

/home/maik/workspace/ecir23-unjudged-documents/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value,ndcg_cut_10 reject,ndcg_cut_10 p-value corrected
7,oracle,1.000000,28.0,0.0,4.594535e-10,True,5.972896e-09
5,bert,0.717930,20.0,9.0,1.010455e-02,False,1.313592e-01
6,formula,0.701332,16.0,13.0,2.654518e-02,False,3.450873e-01
2,dummy,0.677439,16.0,13.0,1.104133e-01,False,1.000000e+00
9,V6-model_all_topic_w0,0.661777,16.0,13.0,2.960805e-01,False,1.000000e+00
3,random,0.658674,17.0,12.0,2.409449e-01,False,1.000000e+00
1,boromir-touche23,0.650130,19.0,11.0,3.612749e-01,False,1.000000e+00
10,V6-model_all_topic_w5,0.628625,15.0,14.0,8.117163e-01,False,1.000000e+00
0,google,0.618670,NaN,NaN,NaN,False,NaN
4,afinn,0.612545,14.0,15.0,8.649907e-01,False,1.000000e+00


In [18]:
load_report('CON', 'ndcg_cut_10')

/home/maik/workspace/ecir23-unjudged-documents/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value,ndcg_cut_10 reject,ndcg_cut_10 p-value corrected
7,oracle,0.923982,30.0,0.0,1.099686e-19,True,1.429592e-18
1,boromir-touche23,0.253411,18.0,10.0,1.025575e-01,False,1.000000e+00
5,bert,0.246923,20.0,10.0,7.230263e-02,False,9.399341e-01
10,V6-model_all_topic_w5,0.242584,18.0,8.0,5.777903e-02,False,7.511274e-01
9,V6-model_all_topic_w0,0.210703,17.0,11.0,3.344683e-01,False,1.000000e+00
4,afinn,0.203745,16.0,11.0,4.885242e-01,False,1.000000e+00
6,formula,0.196157,14.0,12.0,5.809207e-01,False,1.000000e+00
2,dummy,0.195056,16.0,12.0,6.189308e-01,False,1.000000e+00
11,V8-model_all_topic,0.182068,18.0,9.0,8.690045e-01,False,1.000000e+00
8,V3-model_1,0.179505,12.0,12.0,8.910344e-01,False,1.000000e+00


In [19]:
load_report('ONTOPIC', 'ndcg_cut_10')

/home/maik/workspace/ecir23-unjudged-documents/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value,ndcg_cut_10 reject,ndcg_cut_10 p-value corrected
7,oracle,1.000000,24.0,0.0,0.000002,True,0.000021
3,random,0.917632,17.0,8.0,0.021312,False,0.277054
2,dummy,0.913244,17.0,9.0,0.027756,False,0.360826
5,bert,0.910986,16.0,9.0,0.023198,False,0.301576
9,V6-model_all_topic_w0,0.909476,17.0,9.0,0.042492,False,0.552396
6,formula,0.901319,16.0,10.0,0.039568,False,0.514378
1,boromir-touche23,0.896755,15.0,10.0,0.082797,False,1.000000
10,V6-model_all_topic_w5,0.893703,17.0,9.0,0.153663,False,1.000000
8,V3-model_1,0.885314,15.0,10.0,0.140424,False,1.000000
4,afinn,0.885099,17.0,10.0,0.139966,False,1.000000


### Evaluation nDCG@5

In [20]:
load_report('PRO', 'ndcg_cut_5')

/home/maik/workspace/ecir23-unjudged-documents/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


,name,ndcg_cut_5,ndcg_cut_5 +,ndcg_cut_5 -,ndcg_cut_5 p-value,ndcg_cut_5 reject,ndcg_cut_5 p-value corrected
7,oracle,1.000000,26.0,0.0,6.747387e-09,True,8.771603e-08
5,bert,0.719356,18.0,9.0,4.480012e-02,False,5.824016e-01
6,formula,0.694508,16.0,10.0,1.649737e-01,False,1.000000e+00
1,boromir-touche23,0.685982,16.0,12.0,1.435170e-01,False,1.000000e+00
2,dummy,0.674001,17.0,11.0,2.955085e-01,False,1.000000e+00
10,V6-model_all_topic_w5,0.662608,15.0,11.0,4.111669e-01,False,1.000000e+00
12,V9-model_all_topic,0.656439,15.0,13.0,4.304924e-01,False,1.000000e+00
9,V6-model_all_topic_w0,0.648135,14.0,13.0,5.684454e-01,False,1.000000e+00
3,random,0.632710,15.0,10.0,7.022114e-01,False,1.000000e+00
4,afinn,0.620702,12.0,16.0,9.118888e-01,False,1.000000e+00


In [21]:
load_report('CON', 'ndcg_cut_5')

/home/maik/workspace/ecir23-unjudged-documents/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


,name,ndcg_cut_5,ndcg_cut_5 +,ndcg_cut_5 -,ndcg_cut_5 p-value,ndcg_cut_5 reject,ndcg_cut_5 p-value corrected
7,oracle,0.945136,29.0,0.0,5.585033e-16,True,7.260543e-15
10,V6-model_all_topic_w5,0.261281,13.0,8.0,2.056678e-01,False,1.000000e+00
5,bert,0.217228,14.0,11.0,7.386162e-01,False,1.000000e+00
1,boromir-touche23,0.215080,12.0,12.0,8.061944e-01,False,1.000000e+00
11,V8-model_all_topic,0.214848,14.0,10.0,8.292244e-01,False,1.000000e+00
6,formula,0.211495,13.0,10.0,8.589605e-01,False,1.000000e+00
12,V9-model_all_topic,0.202154,8.0,11.0,9.917279e-01,False,1.000000e+00
0,google,0.201644,NaN,NaN,NaN,False,NaN
9,V6-model_all_topic_w0,0.176333,9.0,12.0,6.058586e-01,False,1.000000e+00
2,dummy,0.175452,10.0,12.0,6.381958e-01,False,1.000000e+00


In [22]:
load_report('ONTOPIC', 'ndcg_cut_5')

/home/maik/workspace/ecir23-unjudged-documents/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


,name,ndcg_cut_5,ndcg_cut_5 +,ndcg_cut_5 -,ndcg_cut_5 p-value,ndcg_cut_5 reject,ndcg_cut_5 p-value corrected
7,oracle,1.000000,15.0,0.0,0.000743,True,0.009660
5,bert,0.917092,13.0,5.0,0.122846,False,1.000000
3,random,0.916174,13.0,5.0,0.151717,False,1.000000
2,dummy,0.914199,13.0,6.0,0.160096,False,1.000000
9,V6-model_all_topic_w0,0.911594,13.0,7.0,0.225945,False,1.000000
6,formula,0.909531,12.0,5.0,0.167139,False,1.000000
1,boromir-touche23,0.909196,10.0,9.0,0.285577,False,1.000000
8,V3-model_1,0.896256,12.0,6.0,0.348500,False,1.000000
10,V6-model_all_topic_w5,0.892238,12.0,7.0,0.530021,False,1.000000
4,afinn,0.886719,12.0,6.0,0.497547,False,1.000000


### Test various measures

In [23]:
load_report('CON', 'Rprec')

/home/maik/workspace/ecir23-unjudged-documents/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


,name,Rprec,Rprec +,Rprec -,Rprec p-value,Rprec reject,Rprec p-value corrected
7,oracle,0.583001,30.0,0.0,2.729715e-11,True,3.548630e-10
1,boromir-touche23,0.148586,19.0,6.0,9.012128e-03,False,1.171577e-01
10,V6-model_all_topic_w5,0.127064,18.0,5.0,1.793807e-02,False,2.331949e-01
5,bert,0.123816,18.0,6.0,4.459797e-02,False,5.797737e-01
9,V6-model_all_topic_w0,0.111398,13.0,6.0,5.019196e-02,False,6.524955e-01
4,afinn,0.107881,11.0,8.0,2.009507e-01,False,1.000000e+00
8,V3-model_1,0.099971,9.0,7.0,2.665608e-01,False,1.000000e+00
11,V8-model_all_topic,0.092088,16.0,10.0,4.123296e-01,False,1.000000e+00
2,dummy,0.087734,13.0,10.0,5.230354e-01,False,1.000000e+00
3,random,0.087357,9.0,10.0,5.859933e-01,False,1.000000e+00


In [24]:
load_report('CON', 'map')

/home/maik/workspace/ecir23-unjudged-documents/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


,name,map,map +,map -,map p-value,map reject,map p-value corrected
7,oracle,0.583607,30.0,0.0,6.035878e-12,True,7.846642e-11
1,boromir-touche23,0.076442,17.0,11.0,1.664866e-01,False,1.000000e+00
5,bert,0.067165,19.0,10.0,1.582054e-01,False,1.000000e+00
10,V6-model_all_topic_w5,0.062586,18.0,8.0,1.455579e-01,False,1.000000e+00
4,afinn,0.057519,15.0,12.0,3.896967e-01,False,1.000000e+00
9,V6-model_all_topic_w0,0.052025,17.0,11.0,4.700898e-01,False,1.000000e+00
12,V9-model_all_topic,0.047969,10.0,15.0,7.535820e-01,False,1.000000e+00
3,random,0.046349,13.0,13.0,7.915391e-01,False,1.000000e+00
11,V8-model_all_topic,0.045661,18.0,9.0,7.937385e-01,False,1.000000e+00
2,dummy,0.043108,16.0,12.0,9.087765e-01,False,1.000000e+00
